In [26]:
import pandas as pd
import numpy as np
import pendulum
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

In [27]:
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest 
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetStatus
from alpaca.trading.enums import AssetClass
from alpaca.data.enums import Adjustment
from alpaca.data.timeframe import TimeFrameUnit

In [28]:
# Environment Variables
load_dotenv()

api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")

In [29]:
# clients
data_client = StockHistoricalDataClient(api_key,secret_key)
trading_client = TradingClient(api_key,secret_key)

In [30]:
# Parameters
# start = pendulum.datetime(2016,1,1)
# end = start.add(years=2)

end = pendulum.datetime(2024,7,11,8,tz='UTC')
start = pendulum.datetime(2024,7,10,8,tz='UTC')


In [31]:
# Get all current assets
asset_request = GetAssetsRequest(
    status = AssetStatus.ACTIVE,
    asset_class = AssetClass.US_EQUITY,
)

all_assets = trading_client.get_all_assets(asset_request)

all_assets = [vars(asset) for asset in all_assets] # Convert objects to dicts

In [32]:
len(all_assets)

11724

In [33]:
df = pd.DataFrame(data=all_assets)

df = df[df['tradable'] & df['fractionable'] & df['shortable']]

df.to_csv("assets_test.csv",index=False)

df

,id,asset_class,exchange,symbol,name,status,tradable,marginable,shortable,easy_to_borrow,fractionable,min_order_size,min_trade_increment,price_increment,maintenance_margin_requirement,attributes
481,9f45f86e-282a-4d47-9d6c-b0ff18fa32c6,AssetClass.US_EQUITY,AssetExchange.ARCA,VNQ,Vanguard Real Estate ETF,AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"
484,3a735b55-d864-4e95-bbb6-a021033f67ec,AssetClass.US_EQUITY,AssetExchange.NASDAQ,UPBD,"Upbound Group, Inc. Common Stock",AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"
488,687076b9-a1af-4ede-895f-7dad4133f542,AssetClass.US_EQUITY,AssetExchange.NYSE,TREX,"Trex Company, Inc.",AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"
490,caa6bd02-518a-42c4-836b-2b453493184b,AssetClass.US_EQUITY,AssetExchange.NASDAQ,TREE,"LendingTree, Inc. Common Stock",AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"
495,981f70e6-3263-46df-ae90-283910e259c8,AssetClass.US_EQUITY,AssetExchange.NYSE,SYF,SYNCHRONY FINANCIAL,AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11713,608fe967-e866-494c-b7e8-df618346dbf7,AssetClass.US_EQUITY,AssetExchange.NYSE,ZWS,Zurn Elkay Water Solutions Corporation,AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"
11714,6f0b4ff5-1a47-4629-94fb-bae8671fa435,AssetClass.US_EQUITY,AssetExchange.NASDAQ,ZVRA,"Zevra Therapeutics, Inc. Common Stock",AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,100.0,"[fractional_eh_enabled, has_options]"
11717,7f76608d-e87b-4dc6-b018-a30655c0f15b,AssetClass.US_EQUITY,AssetExchange.OTC,ZURVY,Zurich Insurance Group Limited Sponsored Ameri...,AssetStatus.ACTIVE,True,False,True,True,True,None,None,None,100.0,[]
11718,2315d972-1110-454f-bc7f-82b36f3861d2,AssetClass.US_EQUITY,AssetExchange.NYSE,ZIM,ZIM Integrated Shipping Services Ltd.,AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"


In [34]:
symbols = df['symbol'].to_list()

In [35]:
# Get previous days market data
bars_request = StockBarsRequest(
    symbol_or_symbols=symbols,
    timeframe=TimeFrame(1,TimeFrameUnit.Day),
    start=start,
    end=end,
    adjustment=Adjustment.ALL,
)

bars = data_client.get_stock_bars(bars_request)

In [36]:
df = bars.df

df = df.reset_index()

df

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AMED,2024-07-11 04:00:00+00:00,98.18,98.46,98.085,98.320,128595.0,2000.0,98.271353
1,BFS,2024-07-11 04:00:00+00:00,37.97,38.39,37.770,38.070,8882.0,323.0,38.141928
2,CASS,2024-07-11 04:00:00+00:00,41.12,42.00,40.660,42.000,7898.0,366.0,41.321959
3,AMWD,2024-07-11 04:00:00+00:00,79.44,82.38,79.440,82.320,57546.0,1153.0,80.459910
4,COHR,2024-07-11 04:00:00+00:00,77.28,78.20,76.630,77.070,241935.0,4578.0,77.474574
...,...,...,...,...,...,...,...,...,...
3720,WST,2024-07-11 04:00:00+00:00,324.66,334.20,324.050,332.095,84293.0,2695.0,331.599801
3721,SPGI,2024-07-11 04:00:00+00:00,466.88,471.00,466.880,469.970,218552.0,7588.0,469.512308
3722,VIAV,2024-07-11 04:00:00+00:00,7.25,7.29,7.160,7.220,348545.0,2482.0,7.244021
3723,WIX,2024-07-11 04:00:00+00:00,163.25,164.58,159.490,161.950,178056.0,2617.0,162.343167
